# Build model V1

# Net Data
 Testing
 
 batch=1
 
 subdivisions=1
 
 Training
 
 batch=64
 
 subdivisions=16
 
 width=608
 
 height=608
 
 channels=3
 
 momentum=0.9
 
 decay=0.0005
 
 angle=0
 
 saturation = 1.5
 
 exposure = 1.5
 
 hue=.1

 
 learning_rate=0.001
 
 burn_in=1000
 
 max_batches = 500200
 
 policy=steps
 
 steps=400000,450000
 
 scales=.1,.1



In [1]:
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Dense, Conv2D, LeakyReLU, ZeroPadding2D
from keras.layers import concatenate
from keras.activations import linear
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adagrad
from keras.utils import plot_model

Using TensorFlow backend.


In [3]:


# Conv
# Layer 1
inputs = Input(shape=(300,300,3))
lay1_1 = Conv2D(32, 3, strides=(1, 1), padding='same')(inputs)
lay1_2 = BatchNormalization(epsilon=0.001)(lay1_1)
lay1_3 = LeakyReLU(alpha=0.1)(lay1_2)

# Downsample

# Conv
# Layer 2
lay2_1 = ZeroPadding2D(((1,0),(1,0)))(lay1_3)
lay2_2 = Conv2D(64, 3, strides=(2, 2), padding='valid')(lay2_1)
lay2_3 = BatchNormalization(epsilon=0.001)(lay2_2)
lay2_4 = LeakyReLU(alpha=0.1)(lay2_3)

# Conv
# Layer 3

lay3_1 = Conv2D(32, 1, strides=(1, 1), padding='same')(lay2_4)
lay3_2 = BatchNormalization(epsilon=0.001)(lay3_1)
lay3_3 = LeakyReLU(alpha=0.1)(lay3_2)

# Conv
# Layer 4

lay4_1 = Conv2D(64, 3, strides=(1, 1), padding='same')(lay3_3)
lay4_2 = BatchNormalization(epsilon=0.001)(lay4_1)
lay4_3 = LeakyReLU(alpha=0.1)(lay4_2)

# Shortcut -> Conv
# Layer 5

lay5_0 = concatenate([lay2_4,lay4_3], axis=-1)
lay5_1 = Conv2D(32, 1, strides=(1, 1), padding='same')(lay5_0)
lay5_2 = linear(lay5_1)

# Downsample

# Conv
# Layer 6

lay6_1 = ZeroPadding2D(((1,0),(1,0)))(lay5_2)
lay6_2 = Conv2D(128, 3, strides=(2, 2), padding='valid')(lay6_1)
lay6_3 = BatchNormalization(epsilon=0.001)(lay6_2)
lay6_4 = LeakyReLU(alpha=0.1)(lay6_3)

# Conv
# Layer 7

lay7_1 = Conv2D(64, 1, strides=(1, 1), padding='same')(lay6_4)
lay7_2 = BatchNormalization(epsilon=0.001)(lay7_1)
lay7_3 = LeakyReLU(alpha=0.1)(lay7_2)

# Conv
# Layer 8

lay8_1 = Conv2D(128, 3, strides=(1, 1), padding='same')(lay7_3)
lay8_2 = BatchNormalization(epsilon=0.001)(lay8_1)
lay8_3 = LeakyReLU(alpha=0.1)(lay8_2)

# Shortcut -> Conv
# Layer 9
lay9_0 = concatenate([lay6_3,lay8_3], axis=-1)
lay9_1 = Conv2D(64, 1, strides=(1, 1), padding='same')(lay9_0)
lay9_2 = linear(lay9_1)

# Conv
# Layer 10

lay10_1 = Conv2D(64, 1, strides=(1, 1), padding='same')(lay9_2)
lay10_2 = BatchNormalization(epsilon=0.001)(lay10_1)
lay10_3 = LeakyReLU(alpha=0.1)(lay10_2)

# Conv
# Layer 11

lay11_1 = Conv2D(128, 3, strides=(1, 1), padding='same')(lay10_3)
lay11_2 = BatchNormalization(epsilon=0.001)(lay11_1)
lay11_3 = LeakyReLU(alpha=0.1)(lay11_2)

# Shortcut -> Conv
# layer 12

lay12_0 = concatenate([lay9_2,lay11_3], axis=-1)
lay12_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay12_0)
lay12_2 = linear(lay9_1)

# Downsample

# Conv 
# Layer 13

lay13_1 = ZeroPadding2D(((1,0),(1,0)))(lay12_2)
lay13_2 = Conv2D(256, 3, strides=(2, 2), padding='valid')(lay13_1)
lay13_3 = BatchNormalization(epsilon=0.001)(lay13_2)
lay13_4 = LeakyReLU(alpha=0.1)(lay13_3)

# Conv
# Layer 14

lay14_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay13_4)
lay14_2 = BatchNormalization(epsilon=0.001)(lay14_1)
lay14_3 = LeakyReLU(alpha=0.1)(lay14_2)

# Conv
# Layer 15

lay15_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay14_3)
lay15_2 = BatchNormalization(epsilon=0.001)(lay15_1)
lay15_3 = LeakyReLU(alpha=0.1)(lay15_2)


# Shortcut -> Conv
# Layer 16

lay16_0 = concatenate([lay13_4,lay15_3], axis=-1)
lay16_1 = Conv2D(64, 1, strides=(1, 1), padding='same')(lay16_0)
lay16_2 = linear(lay16_1)


# Conv
# Layer 17

lay17_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay16_2)
lay17_2 = BatchNormalization(epsilon=0.001)(lay17_1)
lay17_3 = LeakyReLU(alpha=0.1)(lay17_2)

# Conv
# Layer 18

lay18_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay17_3)
lay18_2 = BatchNormalization(epsilon=0.001)(lay18_1)
lay18_3 = LeakyReLU(alpha=0.1)(lay18_2)

# Shortcut -> Conv
# Layer 19

lay19_0 = concatenate([lay16_2,lay18_3], axis=-1)
lay19_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay19_0)
lay19_2 = linear(lay19_1)

# Conv
# Layer 20

lay20_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay19_2)
lay20_2 = BatchNormalization(epsilon=0.001)(lay20_1)
lay20_3 = LeakyReLU(alpha=0.1)(lay20_2)

# Conv
# Layer 21

lay21_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay20_3)
lay21_2 = BatchNormalization(epsilon=0.001)(lay21_1)
lay21_3 = LeakyReLU(alpha=0.1)(lay21_2)

# Shortcut -> Conv
# Layer 22

lay22_0 = concatenate([lay19_2,lay21_3], axis=-1)
lay22_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay22_0)
lay22_2 = linear(lay22_1)

# Conv
# Layer 23

lay23_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay22_2)
lay23_2 = BatchNormalization(epsilon=0.001)(lay23_1)
lay23_3 = LeakyReLU(alpha=0.1)(lay23_2)

# Conv 
# Layer 24 

lay24_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay23_3)
lay24_2 = BatchNormalization(epsilon=0.001)(lay24_1)
lay24_3 = LeakyReLU(alpha=0.1)(lay24_2)


# Shortcut -> Conv
# Layer 25

lay25_0 = concatenate([lay22_2,lay24_3], axis=-1)
lay25_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay25_0)
lay25_2 = linear(lay25_1)


# # Conv
# # Layer 26

# batch_normalize=1
# filters=128
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 27

# batch_normalize=1
# filters=256
# size=3
# stride=1
# pad=1
# activation=leaky

# # Shortcut -> Conv
# # Layer 28

# lay28_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay27_2+lay25_3)
# lay28_2 = linear(lay28_1)

# # Conv
# # Layer 29

# batch_normalize=1
# filters=128
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 30

# batch_normalize=1
# filters=256
# size=3
# stride=1
# pad=1
# activation=leaky

# # Shortcut -> Conv
# # Layer 31

# lay31_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay30_2+lay28_3)
# lay31_2 = linear(lay31_1)

# # Conv
# # Layer 32

# batch_normalize=1
# filters=128
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 33

# batch_normalize=1
# filters=256
# size=3
# stride=1
# pad=1
# activation=leaky

# # Shortcut -> Conv
# # Layer 34

# lay34_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay33_2+lay31_3)
# lay34_2 = linear(lay34_1)

# # Conv
# # Layer 35

# batch_normalize=1
# filters=128
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 36

# batch_normalize=1
# filters=256
# size=3
# stride=1
# pad=1
# activation=leaky

# # Shortcut -> Conv
# # Layer 37

# from=-3
# activation=linear

# # Downsample

# # Conv
# # Layer 38

# batch_normalize=1
# filters=512
# size=3
# stride=2
# pad=1
# activation=leaky

# # Conv
# # Layer 39

# batch_normalize=1
# filters=256
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 40

# batch_normalize=1
# filters=512
# size=3
# stride=1
# pad=1
# activation=leaky

# # Shortcut -> Conv
# # Layer 41

# from=-3
# activation=linear


# # Conv
# # Layer 42

# batch_normalize=1
# filters=256
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 43

# batch_normalize=1
# filters=512
# size=3
# stride=1
# pad=1
# activation=leaky

# # Shortcut -> Conv
# # Layer 44

# from=-3
# activation=linear


# # Conv
# # Layer 45

# batch_normalize=1
# filters=256
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 46

# batch_normalize=1
# filters=512
# size=3
# stride=1
# pad=1
# activation=leaky

# # Shortcut -> Conv
# # Layer 47

# from=-3
# activation=linear


# # Conv
# # Layer 48

# batch_normalize=1
# filters=256
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 49

# batch_normalize=1
# filters=512
# size=3
# stride=1
# pad=1
# activation=leaky

# # Shortcut -> Conv
# # Layer 50

# from=-3
# activation=linear

# # Conv
# # Layer 51

# batch_normalize=1
# filters=256
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 52

# batch_normalize=1
# filters=512
# size=3
# stride=1
# pad=1
# activation=leaky

# # Shortcut -> Conv
# # Layer 53

# from=-3
# activation=linear


# # Conv
# # Layer 54

# batch_normalize=1
# filters=256
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 55

# batch_normalize=1
# filters=512
# size=3
# stride=1
# pad=1
# activation=leaky

# # Shortcut -> Conv
# # Layer 56

# from=-3
# activation=linear


# # Conv
# # Layer 57

# batch_normalize=1
# filters=256
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 58

# batch_normalize=1
# filters=512
# size=3
# stride=1
# pad=1
# activation=leaky

# # Shortcut -> Conv
# # Layer 59

# from=-3
# activation=linear

# # Conv
# # Layer 60

# batch_normalize=1
# filters=256
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 61

# batch_normalize=1
# filters=512
# size=3
# stride=1
# pad=1
# activation=leaky

# # Shortcut -> Conv
# # Layer 62

# from=-3
# activation=linear

# # Downsample

# # Conv
# # Layer 63

# batch_normalize=1
# filters=1024
# size=3
# stride=2
# pad=1
# activation=leaky

# # Conv
# # Layer 64

# batch_normalize=1
# filters=512
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 65

# batch_normalize=1
# filters=1024
# size=3
# stride=1
# pad=1
# activation=leaky

# # Shortcut -> Conv
# # Layer 66

# from=-3
# activation=linear

# # Conv
# # Layer 67

# batch_normalize=1
# filters=512
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 68

# batch_normalize=1
# filters=1024
# size=3
# stride=1
# pad=1
# activation=leaky

# # Shortcut -> Conv
# # Layer 69

# from=-3
# activation=linear

# # Conv
# # Layer 70

# batch_normalize=1
# filters=512
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 71

# batch_normalize=1
# filters=1024
# size=3
# stride=1
# pad=1
# activation=leaky

# # Shortcut -> Conv
# # Layer 72

# from=-3
# activation=linear

# # Conv
# # Layer 73

# batch_normalize=1
# filters=512
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 74

# batch_normalize=1
# filters=1024
# size=3
# stride=1
# pad=1
# activation=leaky

# # Shortcut -> Conv
# # Layer 75

# from=-3
# activation=linear

# ######################

# # Conv
# # Layer 76

# batch_normalize=1
# filters=512
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 77

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=1024
# activation=leaky

# # Conv
# # Layer 78

# batch_normalize=1
# filters=512
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 79

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=1024
# activation=leaky

# # Conv
# # Layer 80

# batch_normalize=1
# filters=512
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 81

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=1024
# activation=leaky

# # Conv
# # Layer 82

# size=1
# stride=1
# pad=1
# filters=255
# activation=linear


# [yolo]
# mask = 6,7,8
# anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
# classes=80
# num=9
# jitter=.3
# ignore_thresh = .7
# truth_thresh = 1
# random=1


# [route]
# layers = -4

# # Conv
# # Layer 83

# batch_normalize=1
# filters=256
# size=1
# stride=1
# pad=1
# activation=leaky

# [upsample]
# stride=2

# [route]
# layers = -1, 61



# # Conv
# # Layer 84

# batch_normalize=1
# filters=256
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 85

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=512
# activation=leaky

# # Conv
# # Layer 86

# batch_normalize=1
# filters=256
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 87

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=512
# activation=leaky

# # Conv
# # Layer 88

# batch_normalize=1
# filters=256
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 89

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=512
# activation=leaky

# # Conv
# # Layer 90

# size=1
# stride=1
# pad=1
# filters=255
# activation=linear


# [yolo]
# mask = 3,4,5
# anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
# classes=80
# num=9
# jitter=.3
# ignore_thresh = .7
# truth_thresh = 1
# random=1



# [route]
# layers = -4

# # Conv
# # Layer 91

# batch_normalize=1
# filters=128
# size=1
# stride=1
# pad=1
# activation=leaky

# [upsample]
# stride=2

# [route]
# layers = -1, 36



# # Conv 92
# batch_normalize=1
# filters=128
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 93

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=256
# activation=leaky

# # Conv
# # Layer 94

# batch_normalize=1
# filters=128
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 95

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=256
# activation=leaky

# # Conv
# # Layer 96

# batch_normalize=1
# filters=128
# size=1
# stride=1
# pad=1
# activation=leaky

# # Conv
# # Layer 97

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=256
# activation=leaky

# # Conv
# # Layer 98

# size=1
# stride=1
# pad=1
# filters=255
# activation=linear


# [yolo]
# mask = 0,1,2
# anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
# classes=80
# num=9
# jitter=.3
# ignore_thresh = .7
# truth_thresh = 1
# random=1



ValueError: Dimensions must be equal, but are 128 and 256 for 'add' (op: 'Add') with input shapes: [?,37,37,128], [?,37,37,256].

In [ ]:
model = Model(inputs=inputs, outputs=lay22_2)
plot_model(model, to_file='model.png',show_shapes='true')